In [2]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 3
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="6-31g", unit='bohr', spin=1, verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

mycc = cc.CCSD(mf)
mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Tue Oct 21 23:47:24 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 3
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000

diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmp00a4qj9f
max_memory 4000 MB (current use 169 MB)
number electrons alpha = 2  beta = 1
Set gradient conv threshold to 3.16228e-05
init E= -1.16526744110419
  alpha nocc = 2  HOMO = -0.149258923222746  LUMO = 0.243598403895771
  beta  nocc = 1  HOMO = -0.541006149691402  LUMO = -0.149258923222746

WARN: system HOMO -0.149258923222746 >= system LUMO -0.149258923222746

cycle= 1 E= -1.57230508734463  delta_E= -0.407  |g|= 0.0957  |ddm|= 0.783
  alpha nocc = 2  HOMO = -0.398572424014128  LUMO = 0.221065308816282
  beta  nocc = 1  HOMO = -0.543435719828569  LUMO = 0.0207078344579285
cycle= 2 E= -1.58418471267616  delta_E= -0.0119  |g|= 0.0512  |ddm|= 0.148
  alpha nocc = 2  HOMO = -0.415518288397719  LUMO = 0.243685936165384
  beta  nocc = 1  HOMO = -0.546348038610519  LUMO = 0.0569711290675732
cycle= 3 E= -1.589337826189

-0.03064223482681718

In [74]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 2,
            'seed': 2,
            'walker_type': 'uhf',
            'trial': 'uhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import pyscf_interface
# from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
# mycc.t1 = t1
# ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
pyscf_interface.prep_afqmc(mycc,options,chol_cut=1e-6)

from ad_afqmc import config, mpi_jax, wavefunctions
import time
from jax import random
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (mpi_jax._prep_afqmc(options))

import h5py
chol_file='FCIDUMP_chol'
with h5py.File(chol_file, "r") as fh5:
    [nelec, nmo, ms, nchol] = fh5["header"]
    h0 = jnp.array(fh5.get("energy_core"))
    h1 = jnp.array(fh5.get("hcore")).reshape(nmo, nmo)
    h1_mod = jnp.array(fh5.get("hcore_mod")).reshape(nmo, nmo)
    chol = jnp.array(fh5.get("chol")).reshape(-1, nmo, nmo)

ham_data['h1_mod'] = h1_mod

#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (2, 1)
# Number of basis functions: 6
# Number of Cholesky vectors: 17
#
# Number of MPI ranks: 1
#
# norb: 6
# nelec: (2, 1)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 2
# seed: 2
# walker_type: uhf
# trial: uhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [77]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

# norb = trial.norb
# noccA, noccB = trial.nelec[0], trial.nelec[1]
# mo_a = np.eye(norb)[:,:noccA]
# s1e = mf.get_ovlp()
# mo_b2a,_ = np.linalg.qr(mf.mo_coeff[0].T @ s1e @ mf.mo_coeff[1])
# # # mo_b2a = mf.mo_coeff[0] @ s1e @ mf.mo_coeff[1]
# mo_b = mo_b2a @ np.eye(norb)[:,:noccB]
# wave_data['mo_coeff'] = [mo_a, mo_b]
# wave_data['mo_b2a'] = mo_b2a
mo_coeff = jnp.array(np.load('mo_coeff.npz')["mo_coeff"])
wave_data['mo_B'] = mo_coeff[1]

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)


prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
print(mf.e_tot - prop_data['e_estimate'])
print('cc energy: ', mycc.e_tot)
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
print(trial._calc_overlap(walker_up,walker_dn,wave_data))
print(trial._calc_energy(walker_up,walker_dn,ham_data,wave_data))

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -1.589755381053025
mf enegry:  -1.5897553966753808
-1.562235585339522e-08
cc energy:  -1.620397631502198
(0.9999999999999998+0j)
(-1.589755381053025+0j)


In [92]:
@partial(jit, static_argnums=0)
def _calc_overlap(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    wave_data: dict,
) -> complex:
    '''unrestricted overlap'''
    olp = jnp.linalg.det(
        wave_data["mo_coeff"][0].T.conj() @ walker_up
    ) * jnp.linalg.det(wave_data["mo_coeff"][1].T.conj() @ walker_dn)

    return olp

@partial(jit, static_argnums=0)
def _calc_green(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    wave_data: dict,
) -> list:
    """Calculates the half green's function.

    Args:
        walker_up: The walker for spin up.
        walker_dn: The walker for spin down.
        wave_data: The trial wave function data.

    Returns:
        green: The half green's function for spin up and spin down.
    """
    green_up = (
        walker_up.dot(jnp.linalg.inv(wave_data["mo_coeff"][0].T.conj() @ walker_up))
    ).T
    green_dn = (
        walker_dn.dot(jnp.linalg.inv(wave_data["mo_coeff"][1].T.conj() @ walker_dn))
    ).T
    return [green_up, green_dn]

@partial(jit, static_argnums=0)
def _ut1t2_walker_olp(
    trial, walker_up: jax.Array, walker_dn: jax.Array, wave_data: dict
) -> complex:
    '''<HF|(t1+t2)|walker> = (t_ia G_ia + t_iajb G_iajb) * <HF|walker>'''
    noccA, t1A, t2AA = trial.nelec[0], wave_data["rot_t1A"], wave_data["rot_t2AA"]
    noccB, t1B, t2BB = trial.nelec[1], wave_data["rot_t1B"], wave_data["rot_t2BB"]
    t2AB = wave_data["rot_t2AB"]
    # convert trial_mo_dn into beta mo_basis
    # wave_data["mo_B"] = <psi_a|psi_b>
    mo_B = wave_data["mo_B"].T @ wave_data["mo_coeff"][1]
    # convert walker_dn into beta mo_basis
    walker_dn_B = wave_data["mo_B"].T @ walker_dn
    green_a = (
        walker_up.dot(jnp.linalg.inv(wave_data["mo_coeff"][0].T.conj() @ walker_up))
    ).T
    # s.t. green_b in beta basis thus can be contracted with beta amplitude
    green_b = (
        walker_dn_B.dot(jnp.linalg.inv(mo_B.T.conj() @ walker_dn_B))
    ).T
    green_a, green_b = green_a[:, noccA:], green_b[:, noccB:]
    o0 = _calc_overlap(trial,walker_up,walker_dn,wave_data)
    o1 = jnp.einsum("ia,ia", t1A, green_a) + jnp.einsum("ia,ia", t1B, green_b)
    o2 = (
        0.5 * jnp.einsum("iajb, ia, jb", t2AA, green_a, green_a)
        + 0.5 * jnp.einsum("iajb, ia, jb", t2BB, green_b, green_b)
        + jnp.einsum("iajb, ia, jb", t2AB, green_a, green_b)
    )
    return (o1 + o2) * o0

In [95]:
from jax import lax

@partial(jit, static_argnums=5)
def _ut1t2_exp1(x: float, h1_mod: jax.Array, walker_up: jax.Array,
                    walker_dn: jax.Array, wave_data: dict, trial):
    '''
    unrestricted t_ia <psi_i^a|exp(x*h1_mod)|walker>/<HF|walker>
    '''
    walker_up_1x = walker_up + x * h1_mod.dot(walker_up)
    walker_dn_1x = walker_dn + x * h1_mod.dot(walker_dn)
    
    olp = _ut1t2_walker_olp(trial,walker_up_1x, walker_dn_1x, wave_data)
    o0 = _calc_overlap(trial,walker_up,walker_dn,wave_data)

    return olp/o0

@partial(jit, static_argnums=5)
def _ut1t2_exp2(x: float, chol_i: jax.Array, walker_up: jax.Array,
                   walker_dn: jax.Array, wave_data: dict, trial) -> complex:
    '''
    t_ia <psi_i^a|exp(x*h2_mod)|walker>/<HF|walker>
    '''

    walker_up_2x = (
        walker_up
        + x * chol_i.dot(walker_up)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_up))
    )
    walker_dn_2x = (
        walker_dn
        + x * chol_i.dot(walker_dn)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_dn))
    )
    
    olp = _ut1t2_walker_olp(trial, walker_up_2x,walker_dn_2x,wave_data)
    o0 = _calc_overlap(trial,walker_up,walker_dn,wave_data)
    
    return olp/o0

@partial(jit, static_argnums=0)
def _calc_energy_pt(trial, walker_up, walker_dn, ham_data, wave_data):
    '''
    t = <psi|T1+T2|phi>/<psi|phi>
    e0 = <psi|H|phi>/<psi|phi>
    e1 = <psi|(T1+T2)(h1+h2)|phi>/<psi|phi>
    '''

    eps=1e-4

    norb = trial.norb
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # one body
    x = 0.0
    f1 = lambda a: _ut1t2_exp1(a,h1_mod,walker_up,walker_dn,wave_data,trial)
    t, d_exp1 = jvp(f1, [x], [1.0])

    # two body
    # c_ij^ab <psi_ij^ab|phi_2x>/<psi_0|phi>
    def scanned_fun(carry, c):
        eps,walker_up,walker_dn,wave_data = carry
        return carry, _ut1t2_exp2(eps,c,walker_up,walker_dn,wave_data,trial)

    _, exp2_p = lax.scan(scanned_fun, (eps,walker_up,walker_dn,wave_data), chol)
    _, exp2_0 = lax.scan(scanned_fun, (0.0,walker_up,walker_dn,wave_data), chol)
    _, exp2_m = lax.scan(scanned_fun, (-1.0 * eps,walker_up,walker_dn,wave_data), chol)
    d2_exp2 = (exp2_p - 2.0 * exp2_0 + exp2_m) / eps / eps

    e0 = trial._calc_energy(walker_up,walker_dn,ham_data,wave_data)
    e1 = (d_exp1 + jnp.sum(d2_exp2) / 2.0 )

    return jnp.real(t), jnp.real(e0), jnp.real(e1)

In [98]:
@partial(jit, static_argnums=0)
def _ucisd_olp(
    trial, walker_up: jax.Array, walker_dn: jax.Array, wave_data: dict
) -> complex:
    '''<(1+ci1+ci2)psi_0|walker>'''

    noccA, ci1A, ci2AA = trial.nelec[0], wave_data["ci1A"], wave_data["ci2AA"]
    noccB, ci1B, ci2BB = trial.nelec[1], wave_data["ci1B"], wave_data["ci2BB"]
    ci2AB = wave_data["ci2AB"]
    # mo_b2a = wave_data["mo_b2a"]
    moB = wave_data["mo_B"]
    walker_dn_B = moB.T.dot(
            walker_dn[:, :noccB]
        )
    # b2a = <psi_a|psi_b>
    # tranform walker_dn to beta basis
    # walker_dn_B = mo_b2a.T.dot(
    #     walker_dn[:, :noccB]
    # )

    def _green( walker_up: jax.Array, walker_dn: jax.Array):
        green_up = (walker_up.dot(jnp.linalg.inv(walker_up[: walker_up.shape[1], :]))).T
        green_dn = (walker_dn.dot(jnp.linalg.inv(walker_dn[: walker_dn.shape[1], :]))).T
        return [green_up, green_dn]

    GFA, GFB = _green(walker_up, walker_dn_B)

    o0 = jnp.linalg.det(walker_up[:noccA, :]) * jnp.linalg.det(
        walker_dn_B[:noccB, :]
    )

    o1 = jnp.einsum("ia,ia", ci1A, GFA[:, noccA:]) + jnp.einsum(
        "ia,ia", ci1B, GFB[:, noccB:]
    )

    # AA
    o2 = 0.5 * jnp.einsum("iajb, ia, jb", ci2AA, GFA[:, noccA:], GFA[:, noccA:])
    # o2 -= 0.25 * jnp.einsum("iajb, ib, ja", ci2AA, GFA[:, noccA:], GFA[:, noccA:])

    # BB
    o2 += 0.5 * jnp.einsum("iajb, ia, jb", ci2BB, GFB[:, noccB:], GFB[:, noccB:])
    # o2 -= 0.25 * jnp.einsum("iajb, ib, ja", ci2BB, GFB[:, noccB:], GFB[:, noccB:])

    # AB
    o2 += jnp.einsum("iajb, ia, jb", ci2AB, GFA[:, noccA:], GFB[:, noccB:])

    return (1 + o1 + o2) * o0

from jax import lax

@partial(jit, static_argnums=5)
def _ucisd_olp_exp1(x: float, h1_mod: jax.Array, walker_up: jax.Array,
                    walker_dn: jax.Array, wave_data: dict, trial):
    '''
    unrestricted <(1+ci1+ci2)psi_0|exp(x*h1_mod)|walker>
    '''
    walker_up_1x = walker_up + x * h1_mod.dot(walker_up)
    walker_dn_1x = walker_dn + x * h1_mod.dot(walker_dn)
    
    olp = _ucisd_olp(trial, walker_up_1x, walker_dn_1x, wave_data)

    return olp

@partial(jit, static_argnums=5)
def _ucisd_olp_exp2(x: float, chol_i: jax.Array, walker_up: jax.Array,
                   walker_dn: jax.Array, wave_data: dict, trial) -> complex:
    '''
    unrestricted <(1+ci1+ci2)psi_0|exp(x*h2_mod)|walker>
    '''

    walker_up_2x = (
        walker_up
        + x * chol_i.dot(walker_up)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_up))
    )
    walker_dn_2x = (
        walker_dn
        + x * chol_i.dot(walker_dn)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_dn))
    )
    
    olp = _ucisd_olp(trial, walker_up_2x,walker_dn_2x,wave_data)
    # o0 = _calc_overlap(trial,walker_up,walker_dn,wave_data)
    
    return olp

@partial(jit, static_argnums=0)
def _calc_energy_pt2(trial, walker_up, walker_dn, ham_data, wave_data):
    '''
    olp = <(1+c1+c2)HF|walker>
    e12 = <(1+c1+c2)HF|(h1+h2)|walker>
    '''

    eps=1e-4

    norb = trial.norb
    h1_mod = ham_data['h1_mod']
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # one body
    x = 0.0
    f1 = lambda a: _ucisd_olp_exp1(a,h1_mod,walker_up,walker_dn,wave_data,trial)
    olp, d_olp1 = jvp(f1, [x], [1.0])

    # two body
    # c_ij^ab <psi_ij^ab|phi_2x>/<psi_0|phi>
    def scanned_fun(carry, c):
        eps,walker_up,walker_dn,wave_data = carry
        return carry, _ucisd_olp_exp2(eps,c,walker_up,walker_dn,wave_data,trial)

    _, olp2_p = lax.scan(scanned_fun, (eps,walker_up,walker_dn,wave_data), chol)
    _, olp2_0 = lax.scan(scanned_fun, (0.0,walker_up,walker_dn,wave_data), chol)
    _, olp2_m = lax.scan(scanned_fun, (-1.0 * eps,walker_up,walker_dn,wave_data), chol)
    d2_olp2 = (olp2_p - 2.0 * olp2_0 + olp2_m) / eps / eps

    # e0 = trial._calc_energy(walker_up,walker_dn,ham_data,wave_data)
    e12 = (d_olp1 + jnp.sum(d2_olp2) / 2.0 ) / olp
    
    h0 = ham_data['h0']

    # e_ci = h0 + e1#/o012

    return jnp.real(olp), jnp.real(h0 + e12)

In [99]:
ci2aa = mycc.t2[0] + 2 * np.einsum("ia,jb->ijab", mycc.t1[0], mycc.t1[0])
ci2aa = (ci2aa - ci2aa.transpose(0, 1, 3, 2)) / 2
ci2aa = ci2aa.transpose(0, 2, 1, 3)
ci2bb = mycc.t2[2] + 2 * np.einsum("ia,jb->ijab", mycc.t1[1], mycc.t1[1])
ci2bb = (ci2bb - ci2bb.transpose(0, 1, 3, 2)) / 2
ci2bb = ci2bb.transpose(0, 2, 1, 3)
ci2ab = mycc.t2[1] + np.einsum("ia,jb->ijab", mycc.t1[0], mycc.t1[1])
ci2ab = ci2ab.transpose(0, 2, 1, 3)
ci1a = np.array(mycc.t1[0])
ci1b = np.array(mycc.t1[1])

wave_data["ci1A"], wave_data["ci2AA"] = ci1a, ci2aa
wave_data["ci1B"], wave_data["ci2BB"] = ci1b, ci2bb
wave_data["ci2AB"] = ci2ab

walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]

print(_ucisd_olp(trial, walker_up, walker_dn, wave_data))

o,e_ci = _calc_energy_pt2(trial, walker_up, walker_dn, ham_data, wave_data)
print(o,e_ci)
# print(e0+e1-t*(e0-h0))
print(e_ci)

# mo_a_A = wave_data['mo_coeff'][0]
# mo_b_B = wave_data["mo_a2b"] @ wave_data['mo_coeff'][1]
# noccA, noccB = trial.nelec[0], trial.nelec[1]
# wave_data["rot_t1A"] = mo_a_A[:noccA,:noccA].T @ ci1a
# wave_data["rot_t2AA"] = jnp.einsum('ik,jl,kalb->iajb',
#     mo_a_A[:noccA,:noccA].T,mo_a_A[:noccA,:noccA].T,ci2aa)
# wave_data["rot_t1B"] = mo_b_B[:noccB,:noccB].T @ ci1b
# wave_data["rot_t2BB"] = jnp.einsum('ik,jl,kalb->iajb',
#     mo_b_B[:noccB,:noccB].T,mo_b_B[:noccB,:noccB].T,ci2bb)
# wave_data["rot_t2AB"] = jnp.einsum('ik,jl,kalb->iajb',
#     mo_a_A[:noccA,:noccA].T,mo_b_B[:noccB,:noccB].T,ci2ab)

(0.9999999999999998+0j)
0.9999999999999998 -1.620397459632811
-1.620397459632811


In [100]:
print(mycc.e_tot)

-1.620397631502198


In [105]:
t2aa = mycc.t2[0] + 2 * np.einsum("ia,jb->ijab", mycc.t1[0], mycc.t1[0])
t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
t2aa = t2aa.transpose(0, 2, 1, 3)
t2bb = mycc.t2[2] + 2 * np.einsum("ia,jb->ijab", mycc.t1[1], mycc.t1[1])
t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
t2bb = t2bb.transpose(0, 2, 1, 3)
t2ab = mycc.t2[1] + np.einsum("ia,jb->ijab", mycc.t1[0], mycc.t1[1])
t2ab = t2ab.transpose(0, 2, 1, 3)
t1a = np.array(mycc.t1[0])
t1b = np.array(mycc.t1[1])
# wave_data["t1A"], wave_data["t2AA"] = t1a, t2aa
# wave_data["t1B"], wave_data["t2BB"] = t1b, t2bb
# wave_data["t2AB"] = t2ab
mo_a_A = wave_data['mo_coeff'][0]
mo_b_B = wave_data["mo_B"].T @ wave_data['mo_coeff'][1]
noccA, noccB = trial.nelec[0], trial.nelec[1]
wave_data["rot_t1A"] = mo_a_A[:noccA,:noccA].T @ t1a
wave_data["rot_t2AA"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_a_A[:noccA,:noccA].T,mo_a_A[:noccA,:noccA].T,t2aa)
wave_data["rot_t1B"] = mo_b_B[:noccB,:noccB].T @ t1b
wave_data["rot_t2BB"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_b_B[:noccB,:noccB].T,mo_b_B[:noccB,:noccB].T,t2bb)
wave_data["rot_t2AB"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_a_A[:noccA,:noccA].T,mo_b_B[:noccB,:noccB].T,t2ab)

In [32]:
print(mf.e_tot,mycc.e_tot)

-1.5897553966753808 -1.620397631502198


In [106]:
t,e0,e1 = _calc_energy_pt(trial, walker_up, walker_dn, ham_data, wave_data)
print(t,e0,e1)
print(e0+e1-t*(e0-h0))

2.1494194222219255e-18 -1.589755381053025 -0.030642234524638864
-1.6203976155776638


In [57]:
@partial(jit, static_argnums=5)
def _uhf_exp1(
    x: float,
    h1: jax.Array,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    wave_data: dict,
    trial,
) -> jax.Array:
    """Helper function for calculating local energy using AD,
    evaluates < psi_T | exp(x * h1) | walker > to linear order"""
    walker_up_1x = walker_up + x * h1[0].dot(walker_up)
    walker_dn_1x = walker_dn + x * h1[1].dot(walker_dn)
    o1 = _calc_overlap(trial, walker_up_1x, walker_dn_1x, wave_data)
    o0 = _calc_overlap(trial, walker_up, walker_dn, wave_data)
    return o1/o0

@partial(jit, static_argnums=5)
def _uhf_exp2(
    x: float,
    chol_i: jax.Array,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    wave_data: dict,
    trial,
) -> jax.Array:
    """Helper function for calculating local energy using AD,
    evaluates < psi_T | exp(x * chol_i) | walker > to quadratic order"""
    walker_up_2x = (
        walker_up
        + x * chol_i.dot(walker_up)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_up))
    )
    walker_dn_2x = (
        walker_dn
        + x * chol_i.dot(walker_dn)
        + x**2 / 2.0 * chol_i.dot(chol_i.dot(walker_dn))
    )
    o2 = _calc_overlap(trial, walker_up_2x, walker_dn_2x, wave_data)
    o0 = _calc_overlap(trial, walker_up, walker_dn, wave_data)
    return o2/o0

@partial(jit, static_argnums=0)
def _calc_energy0(trial, walker_up, walker_dn, ham_data, wave_data):
    '''
    olp = <HF|walker>
    e12 = <HF|(h1+h2)|walker>
    '''

    eps = 3e-4

    norb = trial.norb
    h1_mod = [ham_data['h1_mod'],ham_data['h1_mod']]
    chol = ham_data["chol"].reshape(-1, norb, norb)

    # one body
    x = 0.0
    f1 = lambda a: _uhf_exp1(a,h1_mod,walker_up,walker_dn,wave_data,trial)
    olp, d_olp1 = jvp(f1, [x], [1.0])

    # two body
    # c_ij^ab <psi_ij^ab|phi_2x>/<psi_0|phi>
    def scanned_fun(carry, c):
        eps,walker_up,walker_dn,wave_data = carry
        return carry, _uhf_exp2(eps,c,walker_up,walker_dn,wave_data,trial)

    _, olp2_p = lax.scan(scanned_fun, (eps,walker_up,walker_dn,wave_data), chol)
    _, olp2_0 = lax.scan(scanned_fun, (0.0,walker_up,walker_dn,wave_data), chol)
    _, olp2_m = lax.scan(scanned_fun, (-1.0 * eps,walker_up,walker_dn,wave_data), chol)
    d2_olp2 = (olp2_p - 2.0 * olp2_0 + olp2_m) / eps / eps

    # e0 = trial._calc_energy(walker_up,walker_dn,ham_data,wave_data)
    e12 = (d_olp1 + jnp.sum(d2_olp2) / 2.0 ) / olp
    
    h0 = ham_data['h0']

    # e_ci = h0 + e1#/o012

    return jnp.real(olp), jnp.real(h0 + e12)

In [58]:
mf.e_tot - _calc_energy0(trial, walker_up, walker_dn, ham_data, wave_data)[1]

Array(-4.46029906e-08, dtype=float64)